# 2.2 Building chains & LangChain Expression Language (LCEL)
(Modified version of lesson 2 of https://www.deeplearning.ai/short-courses/functions-tools-agents-langchain/)
<br/><br/>

## Setup

### Install dependencies

In [ ]:
%pip install python-dotenv~=1.0 docarray~=0.40.0 pypdf~=5.1 --upgrade --quiet
%pip install langchain~=0.3.7 langchain_openai~=0.2.6 langchain_community~=0.3.5 --upgrade --quiet

# If running locally, you can do this instead:
#%pip install -r ../requirements.txt

### Load environment variables

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# If running in Google Colab, you can use this code instead:
# from google.colab import userdata
# os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")

### Setup Models

In [ ]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
api_version = "2024-10-01-preview"
llm = AzureChatOpenAI(deployment_name="gpt-4o", temperature=0.0, api_version=api_version)
embedding_model = AzureOpenAIEmbeddings(model="text-embedding-3-large", api_version=api_version)

## Simple Chain

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic} {topic}"
)

output_parser = StrOutputParser()

In [ ]:
# Build a chain (creates a RunnableSequence)
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"topic": "bears"})

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=embedding_model
)
retriever = vectorstore.as_retriever()

In [ ]:
retriever.invoke("where did harrison work?")

In [ ]:
retriever.invoke("what do bears like to eat")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

### Using a RunnableMap perform parallel actions

In [ ]:
from langchain.schema.runnable import RunnableMap

inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

# Invoking the partial chain to see what we get
inputs.invoke({"question": "where did harrison work?"})

In [ ]:
# Using the inputs (map) to build a chain
chain = inputs | prompt | llm | output_parser

In [ ]:
chain.invoke({"question": "where did harrison work?"})

## Bind - reusing and extending a component (Runnable)

### Bind with tools
Read more here:
* https://python.langchain.com/docs/concepts/tool_calling/#tool-execution
* https://python.langchain.com/docs/how_to/tool_calling/

In [ ]:
#Define a tool from a function
from langchain_core.tools import tool

@tool
def weather_search(airport_code: str) -> str:
    """Search for weather given an airport code"""
    return f"Fetching weather for {airport_code}..."

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)

function_model = llm.bind_tools([weather_search])
# Above we're using a specialized function model that binds the weather_search tool to the model. We can also use the 
# standard bind function like this (more verbose):
#function_model = model.bind(tools=[convert_to_openai_tool(weather_search)], tool_choice="required")

In [ ]:
runnable = prompt | function_model

In [ ]:
result = runnable.invoke({"input": "what is the weather in sf"})

In [ ]:
print(result.tool_calls)

## Fallbacks

In [ ]:
from langchain_core.output_parsers import JsonOutputParser


In [ ]:
simple_model = llm.bind(
    temperature=0, 
    model="gpt-4o-mini"
)
simple_chain = simple_model | JsonOutputParser()

In [ ]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [ ]:
# Invoking the model directly and note the response
simple_model.invoke(challenge)

**Note**: The next line is expected to fail, since the model is too basic.

In [ ]:
simple_chain.invoke(challenge) # EXPECTED TO FAIL

### Try using a more advanced (chat) model instead

In [ ]:
json_model = simple_model.bind(response_format={"type": "json_object"})
json_chain = llm | JsonOutputParser()

In [ ]:
json_chain.invoke(challenge)

### Using a fallback for the simpler chain

In [ ]:
final_chain = simple_chain.with_fallbacks([json_chain])

In [ ]:
final_chain.invoke(challenge)

## Runnable Interface - methods common to all Runnable components

### Chain composition

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

### Invoke - execute a chain or runnable

In [ ]:
chain.invoke({"topic": "bears"})

### Batch - run multiple operations in parallel

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

### Streamed response

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Tell me an elaborate joke about {topic}"
)
chain = prompt | llm | output_parser

for chunk in chain.stream({"topic": "bears"}):
    print(chunk, end="|", flush=True)
    

### There are also async version - read more here:
https://python.langchain.com/docs/how_to/lcel_cheatsheet/